In [1]:
import pandas as pd # dataframes
import numpy as np # arrays
import re # regular expression for strings

# Case Study: How Does a Bike-Share Navigate Speedy Success?
<img src="logo.png" width="200" height="200">

### Introduction

In 2016, <font color=blue>Cyclistic</font> launched a successful bike-share offering. 
Since then, the program has grown to a fleet of <u>5,824 bicycles</u> that are geotracked and locked into a network of <u>692 stations</u> across Chicago. 
The bikes can be unlocked from one station and returned to any other station in the system anytime.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments.
One approach that helped make these things possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. 
<span style='background:yellow'>Customers who purchase single-ride or full-day passes</span> are referred to as <font color=blue>casual riders</font>. 
<span style='background:yellow'>Customers who purchase annual memberships</span> are <font color=blue>Cyclistic members</font>.

Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. 
Although the pricing flexibility helps Cyclistic attract more customers, Moreno believes that maximizing the number of annual members will be key to future growth. 
Rather than creatinga marketing campaign that targets all-new customers, Moreno believes there is a very good chance to convert casual riders into members. 
She notes that casual riders are already aware of the Cyclistic program and have chosen Cyclistic for their mobility needs.

Moreno has set a clear goal: **Design marketing strategies aimed at converting casual riders into annual members**. 
In order to do that, however, the marketing analyst team needs to better understand 

* How do annual members and casual riders use Cyclistic bikes differently?
* Why would casual riders buy Cyclistic annual memberships?
* How can Cyclistic use digital media to influence casual riders to become members?

Moreno and her team are interested in analyzing the Cyclistic <u>historical bike trip data</u> to identify trends.
Here, we focus on the first question:

**How do annual members and casual riders use Cyclistic bikes differently?**

We are tasked to produce <font color=red>within a week</font> a report with the following deliverables:

<input type="checkbox" disabled  /> A clear statement of the business task

<input type="checkbox" disabled  /> A description of all data sources used

<input type="checkbox" disabled  /> Documentation of any cleaning or manipulation of data

<input type="checkbox" disabled  /> A summary of your analysis

<input type="checkbox" disabled  /> Supporting visualizations and key findings

<input type="checkbox" disabled  /> Your top three recommendations based on your analysis


## 1. Ask Phase

The ultimate **business goal** is to <u>convert casual riders into annual members</u>. To this end, we

* Derive usage differences between casual riders and annual members from historical data of bike trips

Subsequently, we share these insights with the marketing analytics team to find

* How knowing these differences might help us convert casual riders into annual members?

Finally, these insights are used to develop a marketing strategy to be presented to and approved by the executive team.

<input type="checkbox" disabled  checked/> A clear statement of the business task

## 2. Prepare Phase

### Import Data

The data to explore how different customer types are using Cyclistic bikes can be found under this [link](https://divvy-tripdata.s3.amazonaws.com/index.html). The data is public and has been made available by Motivate International Inc. under this [license](https://ride.divvybikes.com/data-license-agreement). Users' sensitive data has been excluded from the data.
The data has been processed to remove trips that are taken by staff as they service and inspect the system; and any trips that were below 60 seconds in length (potentially false starts or users trying to re-dock a bike to ensure it was secure).

In the following, we import <span style='background:yellow'>historical trip data for the year 2022</span>.

In [2]:
files = [f'2022{month+1:02d}-divvy-tripdata.csv' for month in range(12)] # csv files
df = pd.concat(map(pd.read_csv, files), ignore_index=True) # load and merge

### First glimpse

In [3]:
m, n = df.shape
print(f'Number of rows: {m} and columns: {n}')
print(df.dtypes)
df.head(5)

Number of rows: 5667717 and columns: 13
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


In [4]:
unique_bikes = df['rideable_type'].unique()
print(f'Unique bike types: {unique_bikes}')
unique_users = df['member_casual'].unique()
print(f'Unique user types: {unique_users}')

Unique bike types: ['electric_bike' 'classic_bike' 'docked_bike']
Unique user types: ['casual' 'member']


### First insights

Each ride has a unique identifier, the <font color=blue>ride_id</font>. It is known <font color=blue>when</font> and <font color=blue>where</font> the ride has started and ended. Furthermore, we know the <font color=blue>type of bike</font> (electric vs classic) being used as well as the <font color=blue>rider type</font> (member vs casual).

* <font color=blue>ride_id</font>: 16-characters-long ride identifier (consistent across all rows?)
* <font color=blue>rideable_type</font>: 'electric_bike' or 'classic_bike' or 'docked_bike'
* <font color=blue>started_at, ended_at</font>: convert strings to datetime format! sort data chronologically (by started_at)!
* <font color=blue>start_station_name || start_station_id || start_lat, start_lng</font>: start station ids (check consistency and keep one identifier)
* <font color=blue>end_station_name || end_station_id || end_lat, start_lng</font>: end station ids (check consistency and keep one identifier)
* <font color=blue>member_casual</font>: 'casual' or 'member'

### Nulls & Duplicates

In [6]:
count_dups = df.duplicated().values.sum()
print(f'Number of duplicated rows: {count_dups}')

Number of duplicated rows: 0


There are no duplicates. Many rides are missing <font color=blue>station_name</font> and <font color=blue>station_id</font> though this information can be recovered (if necessary) from geographical data <font color=blue>lat</font> and <font color=blue>lng</font>. For some rides, there is no information availible regarding the end geo location.

* The data is <font color=red>reliable</font> apart for some information regarding stations which is recoverable
* The data is <font color=red>original</font>, since it has been collected by Cyclistic itself (first-party)
* The data is <font color=red>comprehensive</font> and contains the crucial feature <font color=blue>'member_casual'</font> to answer one of our business questions
* The data is <font color=red>current</font>, since it contains info about the previous year
* The data is <font color=red>cited</font> and has been promoted by google and is availible at kaggle

The data ROCCCs!

<input type="checkbox" disabled  checked/> A description of all data sources used

## 3. Process Phase

### 3.1 ride_id

check consistency

In [7]:
ids_len = df['ride_id'].apply(len)
print(f'How many ride_ids deviate from 16 characters? Answer: {(ids_len != 16).sum()}')

How many ride_ids deviate from 16 characters? Answer: 0


### 3.2 started_at, ended_at

* convert into datetime
* calculate ride <font color=blue>duration</font>
* extract <font color=blue>daytime</font>(morning, evening)
* extract <font color=blue>weekday</font> (Mo,Tu,...) 
* extract <font color=blue>month</font> (Jan,Feb,...)

In [8]:
### convert into datetime
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df = df.sort_values(by=['started_at']).reset_index() # chronological sorting just in case

In [9]:
### ride duration
df['duration'] = df['ended_at'] - df['started_at']
df['duration'] = df['duration'].dt.total_seconds() / 60

In [10]:
### extract daytime
def daytime(hour):
    if (5 <= hour) and (hour < 12):
        return "morning"
    if (12 <= hour) and (hour < 17):
        return "noon"
    if (17 <= hour) and (hour < 21):
        return "evening"
    else:
        return "night"
    
df['started_at_daytime'] = df['started_at'].dt.hour.apply(daytime)

In [11]:
### extract weekday
df['started_at_weekday'] = df['started_at'].dt.day_name().str[:2]

In [12]:
### extract month
df['started_at_month'] = df['started_at'].dt.month_name().str[:3]

### 3.3 start_station, end_station

A posteriori, one can state that the location data has many inconsistencies. 
Ultimately, we want a unique id for a distinct station. 
The (*lat*, *lng*)-pairs are not suitable due to fluctuations for a single location.
The station_name is a street name and might contain several station_ids.
The station_id should be in principle unique, though it is not.

* In a first step, we need to decide what to do about missing end stations
* In a second step, we want to identify all distinct locations by looking at the features: station_name & station_id & (lat, lng)-pairs.
* In a third step, we want to create a dictionary for every station with an average and std of geo location, i.e., <br>
$\{ \text{centroid_id}: [\text{av_lat}, \text{av_lng}, \text{std_lat}, \text{std_lng}] \}$
* In a fourth step, we assign for every ride_id a start_station and an end_station by geo-distance to the nearest centroid.

### step 1

<u>Assumption</u>: a missing geo location for end_station most likely means that a bike has not been returned (stolen or broken). Thus, we decide to remove these entries from our data.

In [18]:
df = df[df['end_lat'].notna()]

### step 2

We create a new dataframe 'stations' with columns (station_name, station_id, lat, lng) from both start-stations and end-stations.

In [281]:
### create a dataframe containing just station names and correponding geo coordinates
df_a = df[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']]
df_b = df[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']]

# common column names
df_a = df_a.rename(lambda x: x[6:], axis='columns')
df_b = df_b.rename(lambda x: x[4:], axis='columns')

# drop nans
df_a = df_a.dropna()
df_b = df_b.dropna()

stations = pd.concat([df_a, df_b], ignore_index=True)
stations.head(5)

,station_name,station_id,lat,lng
0,Michigan Ave & 8th St,623,41.872773,-87.623981
1,Broadway & Waveland Ave,13325,41.949073,-87.648633
2,Clark St & Ida B Wells Dr,TA1305000009,41.875919,-87.631194
3,Michigan Ave & 8th St,623,41.872773,-87.623981
4,Michigan Ave & 8th St,623,41.872773,-87.623981


In [282]:
### Does each station_id has a unique station_name?
grouped = stations.groupby('station_id')['station_name'].unique() # rows=id and cols=list of names
filtered = grouped[grouped.apply(lambda x: len(x) != 1)] # filter out unique mappings
filtered.apply(len).sort_values(ascending = False) # sort by ambiguity

station_id
564             4
570             3
553             3
623             3
546             3
               ..
621             2
620             2
617             2
615             2
chargingstx1    2
Name: station_name, Length: 351, dtype: int64

In [283]:
### Input a check_id below to see ambiguities in station_names
check_id = 'chargingstx1' # try out '564' '631' '13197' 'chargingstx1'
df_grouped = stations[stations['station_id'] == check_id].groupby('station_name')
print(df_grouped[['station_id']].count())
print(df_grouped[['lat', 'lng']].mean())

                                      station_id
station_name                                    
Bissell St & Armitage Ave - Charging           2
Bissell St & Armitage Ave*                 25302
                                            lat        lng
station_name                                              
Bissell St & Armitage Ave - Charging  41.918018 -87.652183
Bissell St & Armitage Ave*            41.918333 -87.652194


#### comment:

We have identified several ambiguities in <font color=blue>station_names</font> listed below:

* **case 1**: replace a parenthesis <font color=green>' (...)'</font> with an empty string <font color=green>''</font> <br>
e.g. Elizabeth (May) St & Fulton St $\Rightarrow$ Elizabeth St & Fulton St
* **case 2**: remove <font color=green>'Public Rack - '</font> from location names as it causes ambiguity <br> 
e.g. Public Rack - Spaulding Ave & Foster Ave $\Rightarrow$ Spaulding Ave & Foster Ave
* **case 3**: remove <font color=green>' Vaccination Site'</font> from location names as it causes ambiguity <br>
e.g. Olive Harvey Vaccination Site $\Rightarrow$ Olive Harvey
* **case 4** remove <font color=green>'\*'</font> from location names since this differentiates between charging and non-charging <br>
e.g. Bissell St & Armitage Ave* $\Rightarrow$ Bissell St & Armitage Ave
* **case 5** remove <font color=green>' - Charging'</font> from location names since this differentiates between charging and non-charging <br>
e.g. Bissell St & Armitage Ave - Charging $\Rightarrow$ Bissell St & Armitage Ave
* **case 6**: split names on <font color=green>' - '</font> and keep the longest split. <br>
e.g. Lafayette Ave & 87th St - NW $\Rightarrow$ Lafayette Ave & 87th St

Subsequenly, we apply a fix.

In [284]:
def clean(name):
    x = str(name)
    
    if '(' in x: # case 1
        regex = r' \(.*\)'
        x = re.sub(regex, '', name)   
    if x.startswith('Public Rack - '): # case 2
        x = x[14:]    
    if x.endswith(' Vaccination Site'): # case 3
        x = x[:-17]
    if x.endswith('*'): # case 4
        x = x[:-1]
    if x.endswith(' - Charging'): # case 5
        x = x[:-11]
    
    if ' - ' in x: # case 6
        split = x.split(' - ')
        return max(split, key=len)
    return x

stations['station_name'] = stations['station_name'].apply(clean)
stations['station_id'] = stations['station_id'].apply(clean)

In [285]:
charging = stations[stations['station_id'].apply(lambda x: x.startswith('charging'))]
charging_grouped = charging.groupby('station_id')['station_name'].unique()
charging_dict = charging_grouped.apply(lambda x: x[0]).to_dict()

In [293]:
reversed(charging_dict)

In [294]:
### problem: chargingstx0 replaces chargingstx07 which should remain fixed
for x in reversed(charging_dict):
    street = charging_dict[x]
    mask1 = stations['station_name'] == street
    mask2 = (stations['station_id'] != x) & (stations['station_id'] != street)
    id_alt = stations[mask1 & mask2]['station_id'].unique()
    if len(id_alt) == 1:
        print(x)
        print(id_alt)
        stations['station_id'] = stations['station_id'].str.replace(x, id_alt[0])

chargingstx5
['TA1307000138']
chargingstx4
['TA1306000015']
chargingstx3
['13053']
chargingstx1
['13059']
chargingstx06
['13332']
chargingstx0
['TA1306000014']


In [295]:
stations[stations['station_id'].apply(lambda x: x.startswith('chargingstx'))]

,station_name,station_id,lat,lng


In [280]:
stations[stations['station_name'] == 'Loomis St & Lexington St'].groupby('station_id').count()
stations[stations['station_name'] == 'Green St & Madison Ave'].groupby('station_id').count()

,station_name,lat,lng
station_id,,,
TA13060000147,989,989,989


#### remark:

In case there are assignement errors in station_names or station_ids, some might appear unique with just a few registered rides. <br>

<u>Assumption</u>: We decide to **remove a station_name or station_id** if **less than 1000 rides during the whole year** have been registered on that station. Importantly, this allows us to determine the average location for every remaining station. <br>

We protocol the number of deleted stations below and apply a mask on our stations.

In [133]:
### how often does each station_name occur?
counts = stations.groupby(['station_name'])['station_name'].count()
counts_filter = counts.loc[lambda x : x < 1000]
print(f'Number of rows removed: {counts_filter.sum()} vs total number of rows {counts.sum()}')
drop_names = counts_filter.keys().values

Number of rows removed: 142918 vs total number of rows 9592531


In [134]:
### how often does each station_id occur?
counts = stations.groupby(['station_id'])['station_id'].count()
counts_filter = counts.loc[lambda x : x < 1000]
print(f'Number of rows removed: {counts_filter.sum()} vs total number of rows {counts.sum()}')
drop_ids = counts_filter.keys().values

Number of rows removed: 139367 vs total number of rows 9592531


In [135]:
mask = stations['station_name'].apply(lambda x: x not in drop_names)
mask = stations['station_id'].apply(lambda x: x not in drop_ids)
stations = stations[mask]

In [136]:
### Does each station_id has a unique station_name?
grouped = stations.groupby('station_id') # rows=id and cols=list of names

In [137]:
grouped[['lat', 'lng']].mean()

,lat,lng
station_id,,
13001,41.883977,-87.624631
13006,41.882672,-87.632537
13008,41.881056,-87.624096
13011,41.879294,-87.639931
13016,41.894333,-87.622793
...,...,...
chargingstx06,41.872172,-87.661539
chargingstx1,41.918333,-87.652194
chargingstx3,41.883598,-87.648604


In [140]:
### Does each station_id has a unique station_name?
grouped = stations.groupby('station_name')['station_id'].unique() # rows=id and cols=list of names
filtered = grouped[grouped.apply(lambda x: len(x) != 1)] # filter out unique mappings
filtered.apply(len).sort_values(ascending = False) # sort by ambiguity

station_name
Bissell St & Armitage Ave     2
Green St & Randolph St        2
Lincoln Ave & Roscoe St       2
Loomis St & Lexington St      2
Morgan St & Lake St           2
Wilton Ave & Diversey Pkwy    2
Name: station_id, dtype: int64

In [141]:
### Input a check_id below to see ambiguities in station_names
for x in filtered.keys().values:
    check_name = x # try out'564' '631' '13197' 'chargingstx1'
    df_grouped = stations[stations['station_name'] == check_name].groupby('station_id')
    print(x)
    print(df_grouped[['station_name']].count())
    print(df_grouped[['lat', 'lng']].mean())
    print(df_grouped[['lat', 'lng']].std())
    print('_____________')

Bissell St & Armitage Ave
              station_name
station_id                
13059                13715
chargingstx1         25304
                    lat        lng
station_id                        
13059         41.918002 -87.652169
chargingstx1  41.918333 -87.652194
                   lat       lng
station_id                      
13059         0.000045  0.000038
chargingstx1  0.000618  0.000503
_____________
Green St & Randolph St
              station_name
station_id                
13053                 9483
chargingstx3         22039
                    lat        lng
station_id                        
13053         41.883207 -87.648747
chargingstx3  41.883598 -87.648604
                   lat       lng
station_id                      
13053         0.000999  0.001003
chargingstx3  0.000259  0.000077
_____________
Lincoln Ave & Roscoe St
              station_name
station_id                
TA1307000138          6996
chargingstx5         14746
                    lat        

In [146]:
stations[stations['station_id'].apply(lambda x: x.startswith('charging'))].groupby('station_id')['station_name'].unique()

station_id
chargingstx0     [Wilton Ave & Diversey Pkwy]
chargingstx06      [Loomis St & Lexington St]
chargingstx1      [Bissell St & Armitage Ave]
chargingstx3         [Green St & Randolph St]
chargingstx4            [Morgan St & Lake St]
chargingstx5        [Lincoln Ave & Roscoe St]
Name: station_name, dtype: object

In [138]:
### Does each station_id has a unique station_name?
grouped = stations.groupby('station_id')['station_name'].unique() # rows=id and cols=list of names
filtered = grouped[grouped.apply(lambda x: len(x) != 1)] # filter out unique mappings
filtered.apply(len).sort_values(ascending = False) # sort by ambiguity

station_id
13053                                  2
13059                                  2
13074                                  2
514                                    2
517                                    2
518                                    2
519                                    2
520                                    2
523                                    2
546                                    2
549                                    2
644                                    2
Hubbard Bike-checking (LBS-WH-TEST)    2
TA1306000015                           2
TA1307000138                           2
Name: station_name, dtype: int64

In [119]:
### Input a check_id below to see ambiguities in station_names
check_id = '549' # try out'564' '631' '13197' 'chargingstx1'
df_grouped = stations[stations['station_id'] == check_id].groupby('station_name')
print(df_grouped[['station_id']].count())
print(df_grouped[['lat', 'lng']].mean())

                             station_id
station_name                           
Laramie Ave & Fullerton Ave         239
Marshfield Ave & 44th St            804
                                   lat        lng
station_name                                     
Laramie Ave & Fullerton Ave  41.920000 -87.760000
Marshfield Ave & 44th St     41.814002 -87.666603


In [139]:
### Input a check_id below to see ambiguities in station_names
for x in filtered.keys().values:
    check_id = x # try out'564' '631' '13197' 'chargingstx1'
    df_grouped = stations[stations['station_id'] == check_id].groupby('station_name')
    print(x)
    print(df_grouped[['station_id']].count())
    print(df_grouped[['lat', 'lng']].mean())
    print(df_grouped[['lat', 'lng']].std())
    print('_____________')

13053
                            station_id
station_name                          
Green St & Randolph St            9483
Green St & Washington Blvd        7678
                                  lat        lng
station_name                                    
Green St & Randolph St      41.883207 -87.648747
Green St & Washington Blvd  41.883181 -87.648748
                                 lat       lng
station_name                                  
Green St & Randolph St      0.000999  0.001003
Green St & Washington Blvd  0.001026  0.000869
_____________
13059
                           station_id
station_name                         
Bissell St & Armitage Ave       13715
Sheridan Rd & Argyle St          7119
                                 lat        lng
station_name                                   
Bissell St & Armitage Ave  41.918002 -87.652169
Sheridan Rd & Argyle St    41.973278 -87.654736
                                lat       lng
station_name                                

Haversine formula to calculate distances between GPS coordiantes

In [ ]:
df_temp[['start_lat', 'start_lng']].mean()
df_temp[['start_lat', 'start_lng']].std()

<input type="checkbox" disabled  checked/> Documentation of any cleaning or manipulation of data